<img src="https://visionseed.youtu.qq.com/docs/assets/markdown-img-hw-a.jpg">

这篇教程将指导你一步步配置环境、训练模型并将模型部署在VisionSeed上

# 环境配置
## 下载代码

In [ ]:
!git clone https://gitee.com/charllechen/yolov3.git

进入工作文件夹。注：每次**"Restart Kernel"**之后都需要重新运行下面的代码进入文件夹

In [ ]:
%cd yolov3

## 下载依赖库
### pip 依赖库

In [ ]:
!pip install -r requirements.txt -i https://mirrors.cloud.tencent.com/pypi/simple

### 配置PyTorch 1.6.0环境

In [ ]:
!wget https://yolov3-1259675134.cos.ap-shanghai.myqcloud.com/torch-1.6.0%2Bcu101-cp37-cp37m-linux_x86_64.whl
!pip install torch-1.6.0+cu101-cp37-cp37m-linux_x86_64.whl
!pip install torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
# 如果返回True并且torch的版本为1.6.0+cu101，则说明安装好了
import torch
print(torch.cuda.is_available())
!pip show torch

## 下载MS COCO 2017数据库
下载大概需要20分钟

In [ ]:
%mkdir ../coco/images -p
!wget https://yolov3-1259675134.cos.ap-shanghai.myqcloud.com/train2017.zip
!unzip -q train2017.zip -d ../coco/images

### 下载标签

In [ ]:
!wget https://yolov3-1259675134.cos.ap-shanghai.myqcloud.com/annotations_trainval2017.zip
!unzip -q annotations_trainval2017.zip -d ../coco

In [ ]:
#删除压缩包[可选]
!rm annotations_trainval2017.zip

### 配置标签
因为训练代码要求标签文件以txt文件的形式存储在一个文件夹中，与图片的文件夹存储在同一个目录下（标签命名为labels，训练图片命名为images)。并且，除了后缀之外，标签的文件名需要与图片的一致

目录例子
- coco
    - images
        - test.jpg
    - labels
        - test.txt

因此，我们需要运行下面的脚本来改变标签的形式

In [ ]:
# out 设置为标签保存的文件夹，文件夹名称可以按需更改
# image_folder 训练图片的文件夹，保持为默认即可
# json 图片标注信息的json，保持为默认即可
# cls 用于训练的标签种类，可以按需更改，不同类别用逗号隔开（逗号后不要留空格）。具体的标签信息可以参考data/coco.yaml下的names

!python ./utils/coco.py --out ../coco/train_person_dog_cat --image_folder ../coco/images/train2017 --json ../coco/annotations/instances_train2017.json  --cls person,cat,dog

随机选择训练集和测试集

In [ ]:
import random
import os
# 标签的文件夹，应该与上一个cell中的out相同
folder = '../coco/train_person_dog_cat'
TRAIN_PER = 0.99 # 训练集和验证集的比例
files = os.listdir(folder)
files = [f for f in files if f.endswith('.jpg')]
print(files[0])
print("[total dataset size]:", len(files))
random.shuffle(files)
cut = int(TRAIN_PER * len(files))
with open(f'{folder}/train.txt', 'w') as f:
    for img in files[:cut]:
        f.write(f"{folder}/{img}\n")
with open(f'{folder}/val.txt', 'w') as f:
    for img in files[cut:]:
        f.write(f"{folder}/{img}\n")
print(f"Train annotations saved at {folder}/train.txt")
print(f"Val annotations saved at {folder}/val.txt")
print("[Train set size]: ", cut)
print("[Val set size]: ", len(files) - cut)

**同时**，我们需要修改一下data/coco.yaml中train和val两个值（13和14行），让他们指向我们刚刚生成的train.txt和val.txt  
![change coco yaml](https://yolov3-1259675134.cos.ap-shanghai.myqcloud.com/change_coco_yaml.png)

In [ ]:
# 下载预训练模型
!wget https://yolov3-1259675134.cos.ap-shanghai.myqcloud.com/yolov3-tiny.pt

# 训练
接下来我们可以训练了！

In [ ]:
%env LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/opt/conda/envs/pytorch_py3/lib

In [ ]:
!python train.py --data coco.yaml --weights yolov3-tiny.pt --epochs 100 --batch-size 256

训练好的结果会保存在./runs/train/的文件夹中

In [ ]:
# 二次训练：需要将weights对应的值改为相应的上次训练的pt文件
!python train.py --data coco.yaml --weights ./runs/train/exp/weights/last.pt --epochs 1 --batch-size 256

## 模型转换
下载依赖库

In [ ]:
!wget https://yolov3-1259675134.cos.ap-shanghai.myqcloud.com/vstk.tar.gz
!tar xzf vstk.tar.gz
!rm vstk.tar.gz

In [ ]:
!sudo yum -y install protobuf-devel leveldb-devel snappy-devel opencv-devel boost-python36-devel hdf5-devel
!sudo yum -y install gflags-devel glog-devel lmdb-devel
!sudo yum -y install openblas-devel

In [ ]:
!pip install onnx termcolor onnxruntime scikit-image==0.15.0 networkx==2.0 opencv-python -i https://mirrors.cloud.tencent.com/pypi/simple

开始模型转换

In [ ]:
%mkdir conversion conversion/orig conversion/stage conversion/final

In [ ]:
#定义常量
import os
trained_model = './runs/train/exp/weights/best.pt' # 想要转换的pt模型的路径
ORIG = './conversion/orig'
STAGE = './conversion/stage'
FINAL = './conversion/final'

IMG = './conversion/orig/input.png' # 用于测试的图片，保持默认即可

model_name = 'person_dog_cat.onnx' # 模型名称（请以.onnx结尾）
TARGET = os.path.join(FINAL, 'custom_'+model_name).replace('onnx', 'blob')

In [ ]:
# 将pt模型转换为onnx模型
from models.export import pt2onnx
pt2onnx(trained_model, [288, 512])
trained_onnx = trained_model.replace('pt', 'onnx')
print('convert pt to onnx at', trained_onnx)

In [ ]:
# 初始化vstk
import os
os.sys.path.append('/home/tione/notebook/yolov3/vstk/onnx2caffe')
from vstk import surgery, onnxToCaffe, generateBlob

In [ ]:
# 将模型从训练环境复制到模型转换环境
from shutil import copyfile

orig_onnx = os.path.join(ORIG, model_name)
copyfile(trained_onnx, orig_onnx)
print(f"copy {trained_onnx} to {orig_onnx}")

In [ ]:
# 对模型进行裁剪
CUT = [88, 106, 52, 85] # 需要裁剪的层数
OUTPUT_LAYER = [87] # 定义输出层
ACT = "sigmoid" # 定义最后一层的激活函数

stage_onnx = orig_onnx.replace(ORIG, STAGE)
surgery(orig_onnx, stage_onnx, CUT, OUTPUT_LAYER, ACT)
print(f"INPUT: {orig_onnx}\nCUT {CUT}\nADD {OUTPUT_LAYER} output layer and {ACT}\OUTPUT: {stage_onnx}")

In [ ]:
# 将onnx文件转换为Caffe
from vstk import onnxToCaffe
%env GLOG_minloglevel=2
stage_prototxt = stage_onnx.replace('onnx', 'prototxt')
stage_caffe = stage_onnx.replace('onnx', 'caffemodel')
onnxToCaffe(stage_onnx, stage_prototxt, stage_caffe, IMG, S=1, M=0)

In [ ]:
# 将Caffe文件转换为VisionSeed可以使用的blob文件
from vstk import generateBlob
generateBlob(stage_prototxt, nshaves = 2, weights = stage_caffe, ma2480 = True)
os.rename('graph', TARGET)
print("success to generate blob at", TARGET)

下载位于./conversion/final/*.blob的模型文件

In [ ]:
print("完成模型训练，撒花~")